In [ ]:
import os
# Change the terminal placement to the folder where the py files  are located  
os.chdir('/Users/ilyeshammouda/Desktop/Ilyes/2A ENSAE/stage/code_stage_2_A_ENSAE/Stage_2A_ENSAE/code') 
# We position ourselves in the directory where the main classes are located.
print(f" You are currently in the directory: {os.getcwd()}")

In [ ]:
from Zoro import *
from functions import *
from Zoro_MD import * 
from help_function import *
from Zoro_new_Z import * 

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy

In [ ]:
# problem set up
n = 2000
s = int(0.1*n)
# noise amplitude
noiseamp = 0.001
#noiseamp = 0 
decay_factor=0.5
x_star=100*(tools.sparse_vector(n=n,s=s))
#x_star=100*(tools.vecteur_sparse_support_S(n=n,s=s))
A=np.random.normal(size=(n,n)) # Simulate a Gaussian Matrix for the norm_with_a_Gaussian_matrix function
#obj_func = SparseQuadric(n, s, noiseamp)
#obj_func=CompressibleQuadric(n=n, decay_factor=decay_factor, noiseamp=noiseamp)
#obj_func=SparseQuadric(n, s, noiseamp)
#obj_func=square_of_the_difference_support_S(n,x_star,noiseamp)
obj_func=norm_with_a_Gaussian_matrix(n,x_star,noiseamp,A)
# Choose initialization
x0    = np.random.randn(n)
x0    =100*x0/np.linalg.norm(x0)
sparsity = s
#sparsity = int(0.1*len(x0)) # This is a decent default, if no better estimate is known. 

# Parameters for ZORO. Defaults are fine in most cases
params = {"step_size":0.001, "delta": 0.0001, "max_cosamp_iter": 10, 
          "cosamp_tol": 0.5,"sparsity": sparsity,
          'M':A,
          "num_samples": int(np.ceil(np.log(len(x0))*sparsity))}


In [ ]:
from help_function import True_grad_norm_with_a_Gaussian_matrix

In [ ]:
#IHT set up 
lamda_IHT=10 # lambda initialisation 
threshold_IHT=1e-15 # lambda threshold
C_IHT=0.9 # The update param for the lambda_k at each itteration.  
step_IHT=1/(3*(n**2))# The step of the IHT algo 
itt_IHT=500  # The maximum number of itterations for the IHT algo

In [ ]:
#ISTA set up 
lamda_ISTA=4 # lambda initialisation 
threshold_ISTA=0.01 # lambda threshold
C_ISTA=0.9 # The update param for the lambda_k at each itteration.  
step_ISTA=1/(3*n) # The step of the IHT algo 
itt_ISTA=40  # The maximum number of itterations for the IHT algo



In [ ]:
# IHT classique set up
itt_IHT_cla=500
setp_IHT_cla=0.05
s_IHT=sparsity

In [ ]:
#Original Version of the Zoro Algo with different algos for estimating the gradient
opt_Co  = ZORO(x0, obj_func, params,algo='CoSaMP', function_budget= int(1e5),x_star=x_star,s=s)
opt_Lasso  = ZORO(x0, obj_func, params,algo='Lasso', function_budget= int(1e5),itt_Lasso=100,tol_Lasso=0.01,s=s)
opt_DLasso  = ZORO(x0, obj_func, params,algo='DLasso', function_budget= int(1e5),itt_Lasso=100,tol_Lasso=0.01,s=s)
opt_IHT  = ZORO(x0, obj_func, params,algo='IHT_Classique', function_budget= int(1e5),s=sparsity,itt_IHT=itt_IHT_cla,step_IHT=setp_IHT_cla)
opt_IHT_ad  = ZORO(x0, obj_func, params,algo='IHT_ad', function_budget= int(1e5),threshold_IHT=threshold_IHT,C_IHT=C_IHT,lamda_IHT=lamda_IHT,itt_IHT=itt_IHT,step_IHT=step_IHT,s=s)
opt_ista_ad  = ZORO(x0, obj_func, params,algo='ISTA_ad', function_budget= int(1e5),threshold_ista=threshold_ISTA,C_ista=C_ISTA,lamda_ista=lamda_ISTA,itt_ista=itt_ISTA,step_ista=step_ISTA,s=s)
# the actual optimization routine
termination=False

In [ ]:
#opt_Co_New_Z  = ZORO_New_Z(x0, obj_func, params,algo='CoSaMP', function_budget= int(1e5),x_star=x_star,s=s)

# Perform the original Zoro algorithm with different methods for estimating the gradient

## Comparing the algorithm's performance by using a fixed Z versus updating Z at each step for gradient estimation.

In [ ]:
performance_log_ZORO_Co_Z,norm_Estimated_Grad_minus_true_list_CO,est_grad_norm_list_CO=opt_Co.Zoro()

In [ ]:
#performance_log_ZORO_Co_Z_New_Z,norm_Estimated_Grad_minus_true_list_CO_New_Z,est_grad_norm_list_CO_New_Z=opt_Co_New_Z.Zoro_New_Z()

In [ ]:
'''
This peace of code is for comparing the performance of the algorithme using a newly generated Z at each step and a fixed Z

mpl.style.use('seaborn')
fig, ax = plt.subplots()
ax.plot(np.array(performance_log_ZORO_Co_Z_New_Z)[:,0],
 ((np.array(performance_log_ZORO_Co_Z_New_Z)[:,1])), linewidth=1, label = "Performance of the Zoro algorithm with a newly generated Z at each step.")
ax.plot(np.array(performance_log_ZORO_Co_Z)[:,0],
 ((np.array(performance_log_ZORO_Co_Z)[:,1])), linewidth=1, label = "Performance of the Zoro algorithm with a fixed  Z")
plt.xlabel('number of queries')
plt.ylabel('$(f(x_{k})-f^{*})$')
leg = ax.legend()
plt.suptitle("Comparative Analysis of ZORO Algorithm Performance with a newly generated Z at each step and a fixed Z where f(x)=$||x-x^{*}||_{[s]}^{2}$)", fontsize=16)
plt.tight_layout()
plt.show()
'''


In [ ]:
'''
This peace of code is for comparing the performance of the algorithme using a newly generated Z at each step and a fixed Z


mpl.style.use('seaborn')
fig, ax = plt.subplots()
ax.plot(np.array(norm_Estimated_Grad_minus_true_list_CO_Z)[:,0],
 ((np.array(norm_Estimated_Grad_minus_true_list_CO_New_Z)[:,1])), linewidth=1, label = "Zoro algorithm with a newly generated Z at each step.")
ax.plot(np.array(norm_Estimated_Grad_minus_true_list_CO)[:,0],
 ((np.array(norm_Estimated_Grad_minus_true_list_CO)[:,1])), linewidth=1, label = 'Zoro algorithm with a fixed  Z')
plt.xlabel('number of queries')
plt.ylabel('|Estimated_Grad(f(x))- True_Grad(f(x))||')
leg = ax.legend()
plt.suptitle("Comparative Analysis of ZORO Algorithm Performance  where f(x)=$||x-x^{*}||_{[s]}^{2}$)", fontsize=16)
plt.tight_layout()
plt.show()
'''

In [ ]:
'''
This peace of code is for comparing the performance of the algorithme using a newly generated Z at each step and a fixed Z


mpl.style.use('seaborn')
fig, ax = plt.subplots()
ax.plot(np.array(est_grad_norm_list_CO_New_Z)[:,0],
 ((np.array(est_grad_norm_list_CO_New_Z)[:,1])), linewidth=1, label = "Zoro algorithm with a newly generated Z at each step.")
ax.plot(np.array(est_grad_norm_list_CO)[:,0],
 ((np.array(est_grad_norm_list_CO)[:,1])), linewidth=1, label = 'Zoro algorithm with a fixed  Z')
plt.xlabel('number of queries')
plt.ylabel('||Estimated_Grad(f(x))||')
leg = ax.legend()
plt.suptitle("Comparative Analysis of ZORO Algorithm Performance with a newly generated Z at each step and a fixed Z where f(x)=$||x-x^{*}||_{[s]}^{2}$)", fontsize=16)
plt.tight_layout()
plt.show()
'''


In [ ]:
performance_log_ZORO_IHT_ad,norm_Estimated_Grad_minus_true_list_IHT_ad,est_grad_norm_list_IHT_ad=opt_IHT_ad.Zoro()

In [ ]:
performance_log_ZORO_IHT,norm_Estimated_Grad_minus_true_list_IHT,est_grad_norm_list_IHT=opt_IHT.Zoro()

In [ ]:
performance_log_ZORO_Lasso,norm_Estimated_Grad_minus_true_list_Lasso,est_grad_norm_list_Lasso=opt_Lasso.Zoro()

In [ ]:
performance_log_ZORO_DLasso,norm_Estimated_Grad_minus_true_list_DLasso,est_grad_norm_list_DLasso=opt_DLasso.Zoro()

In [ ]:
mpl.style.use('seaborn')

fig, axs = plt.subplots(1, 3, figsize=(18, 8))

ax1 = axs[0]
ax1.plot(np.array(performance_log_ZORO_Lasso)[:,0],
 ((np.array(performance_log_ZORO_Lasso)[:,1])), linewidth=1, label = "ZORO WITH Lasso.")

ax1.plot(np.array(performance_log_ZORO_IHT)[:,0],
 ((np.array(performance_log_ZORO_IHT)[:,1])), linewidth=1, label = "ZORO WITH IHT.")

ax1.set_xlabel('number of queries')
ax1.set_ylabel('$(f(x_{k})-f^{*})$')
ax1.set_title('COSAMP VS DLasso')
ax1.legend()



ax2 = axs[1]
ax2.plot(np.array(norm_Estimated_Grad_minus_true_list_Lasso)[:,0],
 ((np.array(norm_Estimated_Grad_minus_true_list_Lasso)[:,1])), linewidth=1, label = "ZORO WITH Lasso.")

ax2.plot(np.array(norm_Estimated_Grad_minus_true_list_IHT)[:,0],
 ((np.array(norm_Estimated_Grad_minus_true_list_IHT)[:,1])), linewidth=1, label = "ZORO WITH IHT.")
ax2.set_xlabel('number of queries')
ax2.set_ylabel('||Estimated_Grad(f(x))- True_Grad(f(x))||')
ax2.set_title('COSAMP VS DLasso')
ax2.legend()




ax3 = axs[2]
ax3.plot(np.array(est_grad_norm_list_Lasso)[:,0],
 ((np.array(est_grad_norm_list_Lasso)[:,1])), linewidth=1, label = "ZORO WITH Lasso.")

ax3.plot(np.array(est_grad_norm_list_IHT)[:,0],
 ((np.array(est_grad_norm_list_IHT)[:,1])), linewidth=1, label = "ZORO WITH DLasso.")
ax3.set_xlabel('number of queries')
ax3.set_ylabel('||Estimated_Grad(f(x))||')
ax3.set_title('COSAMP VS DLasso')
ax3.legend()



plt.suptitle("Comparative Analysis of ZORO Algorithm Performance with Various Gradient Estimation and Descent Methods where f(x)=$||x-x^{*}||_{[s]}^{2}$)", fontsize=16)
plt.tight_layout()
plt.show()

# Perform the Zoro algorithm with MD and different methods for estimating the gradient

In [ ]:
#Original the Zoro Algo using MD and  with different algos for estimating the gradient
opt_IHT_MD  = ZORO_MD(x0, obj_func, params,algo='IHT_Classique', function_budget= int(1e5),s=sparsity,itt_IHT=itt_IHT,x_star=x_star)
opt_Co_MD  = ZORO_MD(x0, obj_func, params,algo='CoSaMP', function_budget= int(1e5),s=sparsity,itt_IHT=itt_IHT,x_star=x_star)
opt_IHT_ad_MD  = ZORO_MD(x0, obj_func, params,algo='IHT_ad', function_budget= int(1e5),s=sparsity,itt_IHT=itt_IHT,x_star=x_star)
opt_Lass_MD  = ZORO_MD(x0, obj_func, params,algo='Lasso', function_budget= int(1e5),itt_Lasso=100,tol_Lasso=0.01)
opt_DLasso_MD  = ZORO_MD(x0, obj_func, params,algo='DLasso', function_budget= int(1e5),itt_Lasso=100,tol_Lasso=0.01)

In [ ]:
performance_log_ZORO_Co_MD,norm_Estimated_Grad_minus_true_list_CO_MD,est_grad_norm_list_CO_MD=opt_Co_MD.Zoro_MD()

In [ ]:
performance_log_ZORO_IHT_MD,norm_Estimated_Grad_minus_true_list_IHT_MD,est_grad_norm_list_IHT_MD=opt_IHT_MD.Zoro_MD()

In [ ]:
performance_log_ZORO_Lasso_MD,norm_Estimated_Grad_minus_true_list_Lasso_MD,est_grad_norm_list_Lasso_MD=opt_Lass_MD.Zoro_MD()

In [ ]:
performance_log_ZORO_IHT_ad_MD,norm_Estimated_Grad_minus_true_list_IHT_ad_MD,est_grad_norm_list_IHT_ad_MD=opt_IHT_ad_MD.Zoro_MD()

In [ ]:
performance_log_ZORO_DLasso_MD,norm_Estimated_Grad_minus_true_list_DLasso_MD,est_grad_norm_list_DLasso_MD=opt_DLasso_MD.Zoro_MD()

In [ ]:
# Transforming the lists to arrays 
performance_log_ZORO_Co_MD = np.array(performance_log_ZORO_Co_MD)
performance_log_ZORO_Co=np.array(performance_log_ZORO_Co)
performance_log_ZORO_IHT = np.array(performance_log_ZORO_IHT)
performance_log_ZORO_IHT_MD = np.array(performance_log_ZORO_IHT_MD)
performance_log_ZORO_Lasso=np.array(performance_log_ZORO_Lasso)
performance_log_ZORO_Lasso_MD=np.array(performance_log_ZORO_Lasso_MD)
performance_log_ZORO_DLasso=np.array(performance_log_ZORO_DLasso)
performance_log_ZORO_DLasso_MD=np.array(performance_log_ZORO_DLasso_MD)
performance_log_ZORO_IHT_ad=np.array(performance_log_ZORO_IHT_ad)
performance_log_ZORO_IHT_ad_MD=np.array(performance_log_ZORO_IHT_ad_MD)

In [ ]:
mpl.style.use('seaborn')

fig, axs = plt.subplots(2, 3, figsize=(20, 20))

ax1 = axs[0, 0]
ax1.plot(np.array(performance_log_ZORO_Co[:,0]), np.array((performance_log_ZORO_Co[:,1])), linewidth=1, label='ZORO WITH COSAMP')
ax1.plot(performance_log_ZORO_Co_MD[:, 0], (performance_log_ZORO_Co_MD[:, 1]), linewidth=1, label='ZORO USING MD WITH COSAMP')
ax1.set_xlabel('number of queries')
ax1.set_ylabel('$(f(x_{k})-f^{*})$')
ax1.set_title('COSAMP')
ax1.legend()

ax2 = axs[0, 1]
ax2.plot(performance_log_ZORO_IHT[:, 0], (performance_log_ZORO_IHT[:, 1]), linewidth=1, label='ZORO WITH IHT')
ax2.plot(performance_log_ZORO_IHT_MD[:, 0], (performance_log_ZORO_IHT_MD[:, 1]), linewidth=1, label='ZORO USING MD WITH IHT')
ax2.set_xlabel('number of queries')
ax2.set_ylabel('$(f(x_{k})-f^{*})$')
ax2.set_title('IHT')
ax2.legend()

ax3 = axs[1, 0]
ax3.plot(performance_log_ZORO_Lasso[:, 0], (performance_log_ZORO_Lasso[:, 1]), linewidth=1, label='ZORO WITH LASSO')
ax3.plot(performance_log_ZORO_Lasso_MD[:, 0], (performance_log_ZORO_Lasso_MD[:, 1]), linewidth=1, label='ZORO USING MD WITH LASSO')
ax3.set_xlabel('number of queries')
ax3.set_ylabel('$(f(x_{k})-f^{*})$')
ax3.set_title('LASSO')
ax3.legend()

ax4 = axs[1, 1]
ax4.plot(performance_log_ZORO_DLasso[:, 0], (performance_log_ZORO_DLasso[:, 1]), linewidth=1, label='ZORO WITH DE-BIASED LASSO')
ax4.plot(performance_log_ZORO_DLasso_MD[:, 0], (performance_log_ZORO_DLasso_MD[:, 1]), linewidth=1, label='ZORO USING MD WITH DE-BIASED LASSO')
ax4.set_xlabel('number of queries')
ax4.set_ylabel('$(f(x_{k})-f^{*})$')
ax4.set_title('DE-BIASED LASSO')
ax4.legend()

ax5 = axs[1, 2]
ax5.plot(performance_log_ZORO_IHT_ad[:, 0], (performance_log_ZORO_IHT_ad[:, 1]), linewidth=1, label='ZORO WITH IHT AD LASSO')
ax5.plot(performance_log_ZORO_IHT_ad_MD[:, 0], (performance_log_ZORO_IHT_ad_MD[:, 1]), linewidth=1, label='ZORO USING MD WITH IHT AD')
ax5.set_xlabel('number of queries')
ax5.set_ylabel('$(f(x_{k})-f^{*})$')
ax5.set_title('IHT AD LASSO')
ax5.legend()

ax6 = axs[0, 2]
ax6.plot(performance_log_ZORO_IHT_ad[:, 0], (performance_log_ZORO_IHT_ad[:, 1]), linewidth=1, label='ZORO WITH IHT AD LASSO')
ax6.plot(np.array(performance_log_ZORO_Co[:,0]), np.array((performance_log_ZORO_Co[:,1])), linewidth=1, label='ZORO WITH COSAMP')
ax6.set_xlabel('number of queries')
ax6.set_ylabel('$(f(x_{k})-f^{*})$')
ax6.set_title('IHT AD LASSO')
ax6.legend()

plt.suptitle("Comparative Analysis of ZORO Algorithm Performance with Various Gradient Estimation and Descent Methods f(x)=$(||M(x-x^{*}||/n)^{2}$)", fontsize=16)
plt.tight_layout()
plt.show()
